<a href="https://colab.research.google.com/github/tomonari-masada/course2022-stats1/blob/main/topic_analysis_with_PLSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PLSI (PLSA) によるトピック分析
* PLSI = probabilistic latent semantic indexing
* PLSA = probabilistic latent semantic analysis 

## データセット

* scikit-learnを使ったテキストデータの扱い方については、下記リンク先を参照。
 * https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [ ]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

* 文書数を減らすために、4つのカテゴリの文書だけを使う。

In [ ]:
categories = ['rec.motorcycles', 'rec.sport.baseball', 'comp.graphics', 'sci.med']

In [ ]:
twenty_train = fetch_20newsgroups(subset='train', categories=categories)

In [ ]:
twenty_train.target_names

In [ ]:
len(twenty_train.data)

In [ ]:
count_vect = CountVectorizer(min_df=20, max_df=0.1, stop_words="english")
X_train_counts = count_vect.fit_transform(twenty_train.data)

In [ ]:
X_train_counts.shape

In [ ]:
type(X_train_counts)

In [ ]:
X_train_counts = X_train_counts.toarray()

* 単語のインデックス

In [ ]:
count_vect.vocabulary_.get('apple')

In [ ]:
count_vect.get_feature_names_out()[224]

* 除去されたストップワード

In [ ]:
print(count_vect.get_stop_words())

## モデルの設定

* $D$: 文書数
* $W$: 語彙数
* $K$: トピック数（これは自分で決める）

In [ ]:
D, W = X_train_counts.shape
K = 10
print(f"{D} documents, {W} different words, and {K} topics")

## モデルのパラメータ
* `q`の形は[D, W, K]
 * $q_{d,w,k}$は、$d$番目の文書で$w$番目の単語が、$K$個のトピックのうち$k$番目のトピックを表現するために使われる確率。
* `theta`の形は[D, K]
 * $\theta_{d,k}$は、$d$番目の文書で、$K$個のトピックのうち$k$番目のトピックを表現するためにいずれかの単語が使われる確率。
* `phi`の形は、ここでの実装上は、[W, K]
 * 授業資料では、$\phi_{k, w}$と添え字が付けられていたので、ここでは転置してある。
 * $\phi_{k,w}$は、$k$番目のトピックを表現するために、$W$種類ある単語のうち$w$番目の単語が使われる確率。


## M step
* モデルパラメータを更新する。

In [ ]:
def m_step(counts, q):
  pseudo_counts = counts[:, :, None] * q
  theta = pseudo_counts.sum(1)
  theta = theta / theta.sum(-1, keepdims=True)
  phi = pseudo_counts.sum(0)
  phi = phi / phi.sum(0, keepdims=True)
  return theta, phi

## E step

* responsibilityを更新する。

In [ ]:
def e_step(theta, phi):
  q = theta[:, None, :] * phi[None, :, :]
  q = q / q.sum(-1, keepdims=True)
  return q

## lower boundの計算
* EMアルゴリズムがlower boundを大きくしていっているかチェックするため。

In [ ]:
def lower_bound(counts, q, theta, phi):
  pseudo_counts = counts[:, :, None] * q
  lb = (pseudo_counts * np.log(theta[:, None, :] + 1e-16)).sum()
  lb += (pseudo_counts * np.log(phi[None, :, :] + 1e-16)).sum()
  lb -= (pseudo_counts * np.log(q + 1e-16)).sum()
  return lb

## 初期化

In [ ]:
q = np.random.randn(D, W, K)
q = np.exp(q) / np.exp(q).sum(-1, keepdims=True)

## EMアルゴリズムの実行

In [ ]:
for i in range(50):
  theta, phi = m_step(X_train_counts, q)
  q = e_step(theta, phi)
  lb = lower_bound(X_train_counts, q, theta, phi)
  print(f"iter {i+1} | lower bound {lb:.4e}")

## トピック語の表示
* 各$k$について、$\phi_{k,w}$が大きい順に単語を拾う。

In [ ]:
topic_words = np.argsort(- phi, axis=0)
for k in range(K):
  print(k, end=' : ')
  for i in range(20):
    print(count_vect.get_feature_names_out()[topic_words[i, k]], end=', ')
  print()

# 発展課題
* ここでは、$q_{d,w,k}$を（文書数）✖️（語彙数）✖️（トピック数）という巨大な配列として実装した。
* しかし、文書数や語彙数が増えると、このサイズの配列をメインメモリ上に用意することはできなくなる。
* $q_{d,w,k}$をどのように実装すれば、この問題を回避できるか？